In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

In [6]:
input_dir = "D:/workDdrive/Deeplearning/xgboost_intro/input"
print(input_dir)

D:/workDdrive/Deeplearning/xgboost_intro/input


In [12]:
df_train = pd.read_csv(f"{input_dir}/train.csv")
print(len(df_train.index))
print(len(df_train.columns))
df_train.head()

61878
95


,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,4,1,0,0,1,6,1,5,0,0,...,0,1,2,0,0,0,0,0,0,Class_1
4,5,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


In [13]:
df_test = pd.read_csv(f"{input_dir}/test.csv")
print(len(df_test.index))
print(len(df_test.columns))
df_test.head()

144368
94


,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93
0,1,0,0,0,0,0,0,0,0,0,...,0,0,11,1,20,0,0,0,0,0
1,2,2,2,14,16,0,0,0,0,0,...,0,0,0,0,0,4,0,0,2,0
2,3,0,1,12,1,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,1
3,4,0,0,0,1,0,0,0,0,0,...,0,3,1,0,0,0,0,0,0,0
4,5,1,0,0,1,0,0,1,2,0,...,0,0,0,0,0,0,0,9,0,0


クラスは9つあり、分布には偏りあり。
xgboostの認識のために、クラスを数値に変換しておきます。

In [14]:
df_train['target'].value_counts()

Class_2    16122
Class_6    14135
Class_8     8464
Class_3     8004
Class_9     4955
Class_7     2839
Class_5     2739
Class_4     2691
Class_1     1929
Name: target, dtype: int64

In [16]:
Class = {u'Class_1':0,
        u'Class_2':1,
        u'Class_3':2,
        u'Class_4':3,
        u'Class_5':4,
        u'Class_6':5,
        u'Class_7':6,
        u'Class_8':7,
        u'Class_9':8,}

モデルトレーニングですが、本家の説明を見ながらモデリングします。
Scikit-Learnラッパーも用意されていて、馴染みのある文法でも記述できますので、慣れている方はそっちでも良いかもしれません。
が、後に出てくる変数重要度を可視化することなど考えて、通常のモデリングを選択しています。

In [17]:
df_train = df_train.replace(Class)

In [19]:
df_train['target']

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
61848    8
61849    8
61850    8
61851    8
61852    8
61853    8
61854    8
61855    8
61856    8
61857    8
61858    8
61859    8
61860    8
61861    8
61862    8
61863    8
61864    8
61865    8
61866    8
61867    8
61868    8
61869    8
61870    8
61871    8
61872    8
61873    8
61874    8
61875    8
61876    8
61877    8
Name: target, Length: 61878, dtype: int64

In [20]:
df_X = df_train.ix[:,1:94]
df_X.head()

C:\anaconda30\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """Entry point for launching an IPython kernel.


,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93
0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,6,1,5,0,0,1,...,22,0,1,2,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0


In [22]:
df_y = df_train["target"]
df_y.head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64

In [25]:
#Split data set to train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size = 0.2, random_state=1)

In [38]:
#Get labels
labels = pd.read_csv(f"{input_dir}/train_label.csv")
feature_names = list(labels)
#len(feature_names)

In [50]:
df_X_train = pd.DataFrame(X_train)
# dtrain = xgb.DMatrix(df_X_train.as_matrix(),
#                      label=y_train.tolist(),
#                     feature_names=feature_names)
dtrain = xgb.DMatrix(df_X_train.as_matrix(),
                     label=y_train.tolist())

In [51]:
df_X_test = pd.DataFrame(X_test)
dtest = xgb.DMatrix(df_X_test.as_matrix())

パラメータに関しては今回はデフォルト値でやってみます。

In [53]:
# Set parameteres
params={"objective": "multi:softprob", 
        "eval_metric": "mlogloss",
        "eta": 0.3, 
        "max_depth": 6, 
        "min_child_weight": 1, 
        "subsample": 1, 
        "colsample_bytree": 1,
        "num_class": 9
        }

記事では、ここでクロスバリデーションの説明をしてますが、その結果をどう使うか的な記述は一切なく、この記事だけでは消化不良。
なので、別のフォーラムから、「XGBOOST and cross validation」なんてのを拾ってきました。

大したこと書いてあるわけじゃないのですが、num_boost_roundは少なくとも200でやって、テストデータのerror rateの一番少ない回数選びなさいよとのことです。
このnum_boost_roundについては、納得出来てない部分もあるので、後ほど少し考察してみます。
とりあえずで、やってみました。133回が良さそうです。

In [54]:
cv = xgb.cv(params,dtrain,num_boost_round=200,nfold=10)

NameError: name 'prin' is not defined

In [55]:
print(cv)

     test-mlogloss-mean  test-mlogloss-std  train-mlogloss-mean  \
0              1.553377           0.008262             1.539462   
1              1.301285           0.010223             1.279902   
2              1.138108           0.012092             1.110979   
3              1.023593           0.012769             0.990949   
4              0.937692           0.012307             0.900568   
5              0.872693           0.012834             0.831458   
6              0.820213           0.012528             0.774874   
7              0.778277           0.012113             0.729057   
8              0.744267           0.012595             0.691350   
9              0.716865           0.012671             0.660201   
10             0.693565           0.012962             0.633344   
11             0.674594           0.012504             0.610857   
12             0.658474           0.012048             0.591268   
13             0.644413           0.011655             0.57410

In [56]:
#Training
bst = xgb.train(params,dtrain,num_boost_round=133)